In [222]:
# Import packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from parse import preprocess

In [223]:
# Function to show classification report for Cross Validation
def classification_report_with_accuracy_score(y_true, y_pred):
    print(classification_report(y_true, y_pred)) # print classification report
    return accuracy_score(y_true, y_pred) # return accuracy score

In [224]:
# Pre-parse the dataset
data = preprocess("rawfile_blood.csv")


####################################################################
Number of Rows of Dataframe:
1123
Number of Columns of Dataframe:
59

####################################################################
Threshold for number of NULLs in a column: 0.1095
Number of Columns before Parsing for Too Many NULLs in a column:
59
Number of Columns after Parsing for Too Many NULLs in a column:
51

Columns Removed:
B1_b5
B4_a1
B4_a3
B4_a4
B4_a6
B4_b1
B4_b3
B5_a1

####################################################################
Number of Rows before Parsing NULLs in data:
1123
Number of Rows after Parsing NULLs in data:
1007

####################################################################
Number of Columns after dropping A1_2, B1_b4, B2_c3, B4_b2 for inconsistent data types:
47


In [225]:
# Initialise counters for each condition
frail = 0
frail_mci = 0
mci = 0
prefrail_mci = 0
prefrail = 0
robust = 0

# Count rows of data for each condition
for i in range(0, len(data)):
	if data.at[i, 'condition'] == 'frail':
		frail += 1
	elif data.at[i, 'condition'] == 'frail_mci':
		frail_mci += 1
	elif data.at[i, 'condition'] == 'mci':
		mci += 1
	elif data.at[i, 'condition'] == 'prefrail_mci':
		prefrail_mci += 1
	elif data.at[i, 'condition'] == 'prefrail':
		prefrail += 1
	elif data.at[i, 'condition'] == 'robust':
		robust += 1
        
# Display number of rows (frequency) for each condition (label)
print("\n####################################################################")
print("Labels with frequencies:")
print("Frail:", frail)
print("Frail + MCI:", frail_mci)
print("MCI:", mci)
print("Prefrail + MCI:", prefrail_mci)
print("Prefrail:", prefrail)
print("Robust:", robust)


####################################################################
Labels with frequencies:
Frail: 7
Frail + MCI: 76
MCI: 133
Prefrail + MCI: 231
Prefrail: 221
Robust: 339


In [226]:
data.head()

,mtag,condition,A1_1,A2_1,A3_1,B1_a,B1_a1,B1_a2,B1_a3,B1_a4,...,B2_d6,B2_d7,B2_d8,B2_d9,B3,B4_a2,B4_a5,B5_a2,B5_a3,B6
0,ME02646,frail,196,24,46.5,121,3.93,0.37,95,31,...,7,12,13,6,0.2,6.0,1.011,1.14,4.1,5.9
1,ME03109,frail,200,23,55.6,142,4.82,0.42,87,30,...,7,20,17,26,3.1,5.0,1.011,3.25,4.6,8.5
2,ME06997,frail,441,20,76.8,105,4.54,0.41,90,30,...,5,16,19,15,1.4,7.0,1.023,2.14,4.0,6.4
3,ME07149,frail,265,16,47.2,122,4.53,0.39,86,27,...,8,24,19,21,2.1,5.5,1.012,1.06,4.7,6.1
4,ME07700,frail,425,14,31.3,124,4.44,0.38,85,28,...,6,20,23,23,6.0,5.5,1.013,1.95,3.8,5.8


In [227]:
data.columns

Index(['mtag', 'condition', 'A1_1', 'A2_1', 'A3_1', 'B1_a', 'B1_a1', 'B1_a2',
       'B1_a3', 'B1_a4', 'B1_a5', 'B1_a6', 'B1_b', 'B1_b1', 'B1_b2', 'B1_b3',
       'B1_c', 'B1_d', 'B2_a1', 'B2_a2', 'B2_a3', 'B2_a4', 'B2_a5', 'B2_b1',
       'B2_b2', 'B2_b3', 'B2_c1', 'B2_c2', 'B2_c4', 'B2_c5', 'B2_c6', 'B2_c7',
       'B2_d1', 'B2_d2', 'B2_d3', 'B2_d4', 'B2_d5', 'B2_d6', 'B2_d7', 'B2_d8',
       'B2_d9', 'B3', 'B4_a2', 'B4_a5', 'B5_a2', 'B5_a3', 'B6'],
      dtype='object')

In [228]:
c = data['condition'].value_counts()
condition = c.index
c

robust          339
prefrail_mci    231
prefrail        221
mci             133
frail_mci        76
frail             7
Name: condition, dtype: int64

In [229]:
for i in range(len(condition)):
    data['condition'].replace(condition[i], i, inplace = True)

data.head(3)

,mtag,condition,A1_1,A2_1,A3_1,B1_a,B1_a1,B1_a2,B1_a3,B1_a4,...,B2_d6,B2_d7,B2_d8,B2_d9,B3,B4_a2,B4_a5,B5_a2,B5_a3,B6
0,ME02646,5,196,24,46.5,121,3.93,0.37,95,31,...,7,12,13,6,0.2,6.0,1.011,1.14,4.1,5.9
1,ME03109,5,200,23,55.6,142,4.82,0.42,87,30,...,7,20,17,26,3.1,5.0,1.011,3.25,4.6,8.5
2,ME06997,5,441,20,76.8,105,4.54,0.41,90,30,...,5,16,19,15,1.4,7.0,1.023,2.14,4.0,6.4


In [230]:
data.tail()

,mtag,condition,A1_1,A2_1,A3_1,B1_a,B1_a1,B1_a2,B1_a3,B1_a4,...,B2_d6,B2_d7,B2_d8,B2_d9,B3,B4_a2,B4_a5,B5_a2,B5_a3,B6
1002,MV00454,0,220,19,67.5,138,4.66,0.42,91,30,...,20,10,17,8,6.6,7.0,1.015,1.29,4.5,6.2
1003,MV00456,0,334,18,51.0,139,4.63,0.42,91,30,...,16,22,35,40,1.0,6.0,1.015,1.88,3.9,5.6
1004,MV00460,0,418,17,61.0,122,4.18,0.38,90,29,...,19,20,23,15,0.4,6.5,1.005,3.58,4.0,5.6
1005,MV00502,0,393,18,43.1,136,4.57,0.43,94,30,...,13,11,22,23,0.7,7.0,1.009,0.92,4.1,6.0
1006,MV00510,0,371,24,55.9,127,4.41,0.40,90,29,...,13,14,16,12,7.5,8.0,1.017,2.45,4.5,6.2


In [231]:
data.columns

Index(['mtag', 'condition', 'A1_1', 'A2_1', 'A3_1', 'B1_a', 'B1_a1', 'B1_a2',
       'B1_a3', 'B1_a4', 'B1_a5', 'B1_a6', 'B1_b', 'B1_b1', 'B1_b2', 'B1_b3',
       'B1_c', 'B1_d', 'B2_a1', 'B2_a2', 'B2_a3', 'B2_a4', 'B2_a5', 'B2_b1',
       'B2_b2', 'B2_b3', 'B2_c1', 'B2_c2', 'B2_c4', 'B2_c5', 'B2_c6', 'B2_c7',
       'B2_d1', 'B2_d2', 'B2_d3', 'B2_d4', 'B2_d5', 'B2_d6', 'B2_d7', 'B2_d8',
       'B2_d9', 'B3', 'B4_a2', 'B4_a5', 'B5_a2', 'B5_a3', 'B6'],
      dtype='object')

In [232]:
y = data['condition']

features = ['A1_1', 'A2_1', 'A3_1', 'B1_a', 'B1_a1', 'B1_a2',
       'B1_a3', 'B1_a4', 'B1_a5', 'B1_a6', 'B1_b', 'B1_b1', 'B1_b2', 'B1_b3',
       'B1_c', 'B1_d', 'B2_a1', 'B2_a2', 'B2_a3', 'B2_a4', 'B2_a5', 'B2_b1',
       'B2_b2', 'B2_b3', 'B2_c1', 'B2_c2', 'B2_c4', 'B2_c5', 'B2_c6', 'B2_c7',
       'B2_d1', 'B2_d2', 'B2_d3', 'B2_d4', 'B2_d5', 'B2_d6', 'B2_d7', 'B2_d8',
       'B2_d9', 'B3', 'B4_a2', 'B4_a5', 'B5_a2', 'B5_a3', 'B6']
X_old = data[features]

In [233]:
X = X_old
# X = StandardScaler().fit_transform(X_old)
# X = MinMaxScaler().fit_transform(X_old)

In [234]:
# Transform the dataset using SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [235]:
y.shape

(2034,)

In [236]:
X.shape

(2034, 45)

In [237]:
# Test 1: Using the entire dataset as both the train and test sets without splitting into separate train and test sets

In [238]:
# Logistic Regression
log_model = LogisticRegression()
log_model.fit(X, y)
print("Logistic Regression:", log_model.score(X, y).round(3))

# Linear Discriminant Analysis
lda_model = LinearDiscriminantAnalysis()
lda_model.fit(X, y)
print("Linear Discriminant Analysis:", lda_model.score(X, y).round(3))

# K-Nearest Neigbors
knn_model = KNeighborsClassifier()
knn_model.fit(X, y)
print("K-Nearest Neigbors:", knn_model.score(X, y).round(3))

# Classification and Regression Trees
cart_model = DecisionTreeClassifier()
cart_model.fit(X, y)
print("Classification and Regression Trees:", cart_model.score(X, y).round(3))

# Gaussian Naive Bayes
gnb_model = GaussianNB()
gnb_model.fit(X, y)
print("Gaussian Naive Bayes:", gnb_model.score(X, y).round(3))

# Support Vector Machines
svm_model = SVC(gamma = 'auto')
svm_model.fit(X, y)
print("Support Vector Machines:", svm_model.score(X, y).round(3))

# Random Forest Classifier
rfc_model = RandomForestClassifier()
rfc_model.fit(X, y)
print("Random Forest Classifier:", rfc_model.score(X, y).round(3))

/Users/shahidan/anaconda3/envs/FYP/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Logistic Regression: 0.424
Linear Discriminant Analysis: 0.52
K-Nearest Neigbors: 0.772
Classification and Regression Trees: 1.0
Gaussian Naive Bayes: 0.486
Support Vector Machines: 1.0
Random Forest Classifier: 1.0


In [239]:
# Test 2: Splitting the dataset into separate train and test sets

In [240]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [241]:
# Logistic Regression
log_model = LogisticRegression()
log_model.fit(X_train, y_train)
print("Logistic Regression:", log_model.score(X_test, y_test).round(3))

# Linear Discriminant Analysis
lda_model = LinearDiscriminantAnalysis()
lda_model.fit(X_train, y_train)
print("Linear Discriminant Analysis:", lda_model.score(X_test, y_test).round(3))

# K-Nearest Neigbors
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
print("K-Nearest Neigbors:", knn_model.score(X_test, y_test).round(3))

# Classification and Regression Trees
cart_model = DecisionTreeClassifier()
cart_model.fit(X_train, y_train)
print("Classification and Regression Trees:", cart_model.score(X_test, y_test).round(3))

# Gaussian Naive Bayes
gnb_model = GaussianNB()
gnb_model.fit(X_train, y_train)
print("Gaussian Naive Bayes:", gnb_model.score(X_test, y_test).round(3))

# Support Vector Machines
svm_model = SVC(gamma = 'auto')
svm_model.fit(X_train, y_train)
print("Support Vector Machines:", svm_model.score(X_test, y_test).round(3))

# Random Forest Classifier
rfc_model = RandomForestClassifier()
rfc_model.fit(X_train, y_train)
print("Random Forest Classifier:", rfc_model.score(X_test, y_test).round(3))

/Users/shahidan/anaconda3/envs/FYP/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Logistic Regression: 0.361
Linear Discriminant Analysis: 0.459
K-Nearest Neigbors: 0.609
Classification and Regression Trees: 0.55
Gaussian Naive Bayes: 0.482
Support Vector Machines: 0.504
Random Forest Classifier: 0.73


In [242]:
# Logistic Regression
log_model = LogisticRegression()
log_model.fit(X, y)
scores = cross_val_score(log_model, X, y, cv=10)
print("Logistic Regression: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

# Linear Discriminant Analysis
lda_model = LinearDiscriminantAnalysis()
lda_model.fit(X, y)
scores = cross_val_score(lda_model, X, y, cv=10)
print("Linear Discriminant Analysis: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

# K-Nearest Neigbors
knn_model = KNeighborsClassifier()
knn_model.fit(X, y)
scores = cross_val_score(knn_model, X, y, cv=10)
print("K-Nearest Neighbors: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

# Classification and Regression Trees
cart_model = DecisionTreeClassifier()
cart_model.fit(X, y)
scores = cross_val_score(cart_model, X, y, cv=10)
print("Classification and Regression Trees: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

# Gaussian Naive Bayes
gnb_model = GaussianNB()
gnb_model.fit(X, y)
scores = cross_val_score(gnb_model, X, y, cv=10)
print("Gaussian Naive Bayes: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

# Support Vector Machines
svm_model = SVC(gamma = 'auto')
svm_model.fit(X, y)
scores = cross_val_score(svm_model, X, y, cv=10)
print("Support Vector Machines: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

# Random Forest Classifier
rfc_model = RandomForestClassifier()
rfc_model.fit(X, y)
scores = cross_val_score(rfc_model, X, y, cv=10)
print("Random Forest Classifier: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

/Users/shahidan/anaconda3/envs/FYP/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/shahidan/anaconda3/envs/FYP/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stabl

Logistic Regression: 0.39 accuracy with a standard deviation of 0.03
Linear Discriminant Analysis: 0.47 accuracy with a standard deviation of 0.04
K-Nearest Neighbors: 0.60 accuracy with a standard deviation of 0.06
Classification and Regression Trees: 0.57 accuracy with a standard deviation of 0.04
Gaussian Naive Bayes: 0.46 accuracy with a standard deviation of 0.03
Support Vector Machines: 0.52 accuracy with a standard deviation of 0.03
Random Forest Classifier: 0.75 accuracy with a standard deviation of 0.07


In [243]:
# Logistic Regression
scores = cross_val_score(log_model, X, y, cv=10, scoring=make_scorer(classification_report_with_accuracy_score))
print(scores)

              precision    recall  f1-score   support

           0       0.23      0.29      0.26        34
           1       0.29      0.15      0.20        34
           2       0.25      0.18      0.21        34
           3       0.26      0.29      0.28        34
           4       0.55      0.35      0.43        34
           5       0.53      0.94      0.68        34

    accuracy                           0.37       204
   macro avg       0.35      0.37      0.34       204
weighted avg       0.35      0.37      0.34       204



/Users/shahidan/anaconda3/envs/FYP/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/shahidan/anaconda3/envs/FYP/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stabl

              precision    recall  f1-score   support

           0       0.47      0.41      0.44        34
           1       0.24      0.12      0.16        34
           2       0.22      0.18      0.20        34
           3       0.24      0.29      0.27        34
           4       0.39      0.32      0.35        34
           5       0.56      1.00      0.72        34

    accuracy                           0.39       204
   macro avg       0.35      0.39      0.35       204
weighted avg       0.35      0.39      0.35       204

              precision    recall  f1-score   support

           0       0.33      0.21      0.25        34
           1       0.32      0.24      0.27        34
           2       0.43      0.26      0.33        34
           3       0.30      0.26      0.28        34
           4       0.32      0.41      0.36        34
           5       0.54      1.00      0.70        34

    accuracy                           0.40       204
   macro avg       0.37

/Users/shahidan/anaconda3/envs/FYP/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/shahidan/anaconda3/envs/FYP/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stabl

              precision    recall  f1-score   support

           0       0.24      0.21      0.22        34
           1       0.23      0.15      0.18        34
           2       0.32      0.18      0.23        34
           3       0.11      0.12      0.11        34
           4       0.44      0.53      0.48        34
           5       0.62      1.00      0.76        34

    accuracy                           0.36       204
   macro avg       0.32      0.36      0.33       204
weighted avg       0.32      0.36      0.33       204

              precision    recall  f1-score   support

           0       0.29      0.27      0.28        33
           1       0.19      0.21      0.20        34
           2       0.38      0.18      0.24        34
           3       0.24      0.24      0.24        34
           4       0.30      0.38      0.33        34
           5       0.71      0.88      0.79        34

    accuracy                           0.36       203
   macro avg       0.35

/Users/shahidan/anaconda3/envs/FYP/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/shahidan/anaconda3/envs/FYP/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stabl

              precision    recall  f1-score   support

           0       0.27      0.24      0.25        34
           1       0.24      0.18      0.20        34
           2       0.41      0.27      0.33        33
           3       0.48      0.47      0.48        34
           4       0.35      0.32      0.34        34
           5       0.55      1.00      0.71        34

    accuracy                           0.41       203
   macro avg       0.38      0.41      0.38       203
weighted avg       0.38      0.41      0.38       203

              precision    recall  f1-score   support

           0       0.39      0.53      0.45        34
           1       0.12      0.09      0.11        33
           2       0.36      0.26      0.31        34
           3       0.54      0.38      0.45        34
           4       0.45      0.38      0.41        34
           5       0.62      1.00      0.76        34

    accuracy                           0.44       203
   macro avg       0.41

/Users/shahidan/anaconda3/envs/FYP/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/shahidan/anaconda3/envs/FYP/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stabl

              precision    recall  f1-score   support

           0       0.39      0.26      0.32        34
           1       0.14      0.09      0.11        34
           2       0.26      0.26      0.26        34
           3       0.24      0.30      0.27        33
           4       0.31      0.26      0.29        34
           5       0.60      0.94      0.74        34

    accuracy                           0.35       203
   macro avg       0.32      0.35      0.33       203
weighted avg       0.32      0.35      0.33       203

              precision    recall  f1-score   support

           0       0.16      0.12      0.14        34
           1       0.28      0.15      0.19        34
           2       0.29      0.24      0.26        34
           3       0.25      0.32      0.28        34
           4       0.44      0.48      0.46        33
           5       0.65      1.00      0.79        34

    accuracy                           0.38       203
   macro avg       0.35

/Users/shahidan/anaconda3/envs/FYP/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [244]:
# Linear Discriminant Analysis
scores = cross_val_score(lda_model, X, y, cv=10, scoring=make_scorer(classification_report_with_accuracy_score))
print(scores)

              precision    recall  f1-score   support

           0       0.33      0.47      0.39        34
           1       0.33      0.18      0.23        34
           2       0.31      0.32      0.31        34
           3       0.39      0.32      0.35        34
           4       0.39      0.26      0.32        34
           5       0.66      0.97      0.79        34

    accuracy                           0.42       204
   macro avg       0.40      0.42      0.40       204
weighted avg       0.40      0.42      0.40       204

              precision    recall  f1-score   support

           0       0.48      0.47      0.48        34
           1       0.33      0.32      0.33        34
           2       0.25      0.29      0.27        34
           3       0.38      0.29      0.33        34
           4       0.40      0.35      0.38        34
           5       0.81      1.00      0.89        34

    accuracy                           0.46       204
   macro avg       0.44

In [245]:
# K-Nearest Neigbors
knn_model = KNeighborsClassifier()
knn_model.fit(X, y)
scores = cross_val_score(knn_model, X, y, cv=10, scoring=make_scorer(classification_report_with_accuracy_score))
print(scores)

              precision    recall  f1-score   support

           0       0.56      0.29      0.38        34
           1       0.37      0.29      0.33        34
           2       0.32      0.29      0.31        34
           3       0.62      0.68      0.65        34
           4       0.66      0.85      0.74        34
           5       0.72      1.00      0.84        34

    accuracy                           0.57       204
   macro avg       0.54      0.57      0.54       204
weighted avg       0.54      0.57      0.54       204

              precision    recall  f1-score   support

           0       0.28      0.24      0.25        34
           1       0.52      0.32      0.40        34
           2       0.38      0.35      0.36        34
           3       0.50      0.59      0.54        34
           4       0.65      0.71      0.68        34
           5       0.73      0.97      0.84        34

    accuracy                           0.53       204
   macro avg       0.51

In [246]:
# Classification and Regression Trees
cart_model = DecisionTreeClassifier()
cart_model.fit(X, y)
scores = cross_val_score(cart_model, X, y, cv=10, scoring=make_scorer(classification_report_with_accuracy_score))
print(scores)

              precision    recall  f1-score   support

           0       0.34      0.32      0.33        34
           1       0.37      0.32      0.34        34
           2       0.37      0.47      0.42        34
           3       0.55      0.50      0.52        34
           4       0.58      0.62      0.60        34
           5       1.00      0.94      0.97        34

    accuracy                           0.53       204
   macro avg       0.54      0.53      0.53       204
weighted avg       0.54      0.53      0.53       204

              precision    recall  f1-score   support

           0       0.27      0.26      0.27        34
           1       0.34      0.32      0.33        34
           2       0.33      0.41      0.37        34
           3       0.40      0.29      0.34        34
           4       0.61      0.65      0.63        34
           5       0.92      0.97      0.94        34

    accuracy                           0.49       204
   macro avg       0.48

In [247]:
# Gaussian Naive Bayes
gnb_model = GaussianNB()
gnb_model.fit(X, y)
scores = cross_val_score(gnb_model, X, y, cv=10, scoring=make_scorer(classification_report_with_accuracy_score))
print(scores)

              precision    recall  f1-score   support

           0       0.33      0.41      0.36        34
           1       0.38      0.24      0.29        34
           2       0.23      0.21      0.22        34
           3       0.32      0.53      0.40        34
           4       0.69      0.32      0.44        34
           5       0.92      1.00      0.96        34

    accuracy                           0.45       204
   macro avg       0.48      0.45      0.44       204
weighted avg       0.48      0.45      0.44       204

              precision    recall  f1-score   support

           0       0.42      0.32      0.37        34
           1       0.26      0.24      0.25        34
           2       0.20      0.18      0.19        34
           3       0.25      0.41      0.31        34
           4       0.38      0.29      0.33        34
           5       0.94      1.00      0.97        34

    accuracy                           0.41       204
   macro avg       0.41

In [248]:
# Support Vector Machines
svm_model = SVC(gamma = 'auto')
svm_model.fit(X, y)
scores = cross_val_score(svm_model, X, y, cv=10, scoring=make_scorer(classification_report_with_accuracy_score))
print(scores)

              precision    recall  f1-score   support

           0       0.27      1.00      0.43        34
           1       1.00      0.15      0.26        34
           2       1.00      0.21      0.34        34
           3       1.00      0.50      0.67        34
           4       1.00      0.59      0.74        34
           5       1.00      0.91      0.95        34

    accuracy                           0.56       204
   macro avg       0.88      0.56      0.56       204
weighted avg       0.88      0.56      0.56       204

              precision    recall  f1-score   support

           0       0.25      1.00      0.40        34
           1       1.00      0.18      0.30        34
           2       1.00      0.24      0.38        34
           3       1.00      0.21      0.34        34
           4       1.00      0.56      0.72        34
           5       1.00      0.82      0.90        34

    accuracy                           0.50       204
   macro avg       0.88

In [249]:
# Random Forest Classifier
rfc_model = RandomForestClassifier()
rfc_model.fit(X, y)
scores = cross_val_score(rfc_model, X, y, cv=10, scoring=make_scorer(classification_report_with_accuracy_score))
print(scores)

              precision    recall  f1-score   support

           0       0.48      0.59      0.53        34
           1       0.72      0.62      0.67        34
           2       0.60      0.44      0.51        34
           3       0.76      0.85      0.81        34
           4       0.89      0.94      0.91        34
           5       0.97      0.97      0.97        34

    accuracy                           0.74       204
   macro avg       0.74      0.74      0.73       204
weighted avg       0.74      0.74      0.73       204

              precision    recall  f1-score   support

           0       0.38      0.50      0.43        34
           1       0.56      0.56      0.56        34
           2       0.50      0.44      0.47        34
           3       0.68      0.50      0.58        34
           4       0.86      0.91      0.89        34
           5       1.00      1.00      1.00        34

    accuracy                           0.65       204
   macro avg       0.66

In [250]:
# # Calculating accuracy metrics for LDA
# lda_model.fit(X_train, y_train)
# lda_pred = lda_model.predict(X_test)

# print('Accuracy Metrics for LDA:\n')
# print(accuracy_score(y_test, lda_pred).round(5), '\n')
# print(confusion_matrix(y_test, lda_pred), '\n')
# print(classification_report(y_test, lda_pred))